#  1 Preparing data

installs

In [ ]:
#%pip install catboost
#%pip install imbalanced_learn==0.11.0
#%pip install imblearn==0.0
#%pip install matplotlib==3.6.2
#%pip install numpy==1.23.5
#%pip install pandas==2.2.0
#%pip install scikit_learn==1.4.0
#%pip install seaborn==0.13.2

## 1.1 Reading Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from catboost import CatBoostClassifier
from catboost import Pool
filepath='./data/catB_train.parquet'
df = pd.read_parquet(filepath)

## 1.2 Data Visualisation

## 1.2 Processing Data

We fill in the target columns that are missing with 0. 1 indicates that the client bought an insurance policy within 3 months and 0 means that the client did not

In [ ]:
df["f_purchase_lh"] = df["f_purchase_lh"].fillna(0)

Here we plot a graph to show the proportion of clients who bought and did not buy within 3 months and we can see that the number is very skewed

In [ ]:
plt.bar([0,1], df['f_purchase_lh'].value_counts(), tick_label=['Did not buy','Bought'])
plt.title('Proportion of clients purchasing insurance or not within 3 months')
plt.xlabel('Clients')
plt.ylabel('Frequency')
plt.show()

In [ ]:
df.replace('None',None,inplace=True)

In [ ]:
df['min_occ_date'].isna().value_counts()

We removed the client id as it is irrelevant for predicting.

In [ ]:
df=df.drop(columns=['clntnum'])

We then created a column which indicates the client's age when they bought the policy

In [ ]:
df['adjusted_first_purchase'] = (datetime.datetime.now()-pd.to_datetime(df['min_occ_date']).fillna(pd.to_datetime(df['min_occ_date']).median())).dt.days/365
df['adjusted_dob'] = (datetime.datetime.now()-pd.to_datetime(df['cltdob_fix']).fillna(pd.to_datetime(df['cltdob_fix']).median())).dt.days/365
df=df.drop(columns=['cltdob_fix', 'min_occ_date'])

In [ ]:
df['client_first_purchase_age']=df['adjusted_dob']-df['adjusted_first_purchase']

This is a histogram of the age for the clients

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(df['adjusted_dob'], bins=20, color='skyblue', edgecolor='black')
plt.title('Histogram of age')
plt.xlabel('Years')
plt.ylabel('Frequency')
plt.show()

This graph shows the histogram of client's age when they first purchased the policy

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(df['client_first_purchase_age'], bins=20, color='skyblue', edgecolor='black')
plt.title('Histogram of client_first_purchase_age')
plt.xlabel('Years')
plt.ylabel('Frequency')
plt.show()

In [ ]:
negative_values = df[df['client_first_purchase_age'] < 0]
negative_values

We have a list of clients from different countries but we see that Singaporean and Malaysian clients are more likely to be in the client list

In [ ]:
df['ctrycode_desc'].value_counts()

We grouped the clients into Singaporeans, Malaysians and others and made them into categorical variables

In [ ]:
df['ctrycode_grouped'] = df['ctrycode_desc'].apply(lambda x: x if x in ['Singapore', 'Malaysia'] else 'Others')
df=df.drop(columns=['ctrycode_desc'])

We see that Malaysians there are 85 Malaysians but none have bought an insurance within the last 3 months

In [ ]:
print(df[df['ctrycode_grouped'] == 'Malaysia'].shape[0])
print(sum(df[df['ctrycode_grouped'] == 'Malaysia']['f_purchase_lh']))

Furthermore we see that there are 118 people who are not Singaporeans or Malaysians and none have bought an insurance within the last 3 months

In [ ]:
print(df[df['ctrycode_grouped'].apply(lambda x:x not in ['Malaysia','Singapore'])].shape[0])
print(sum(df[df['ctrycode_grouped'].apply(lambda x:x  not in ['Malaysia','Singapore'])]['f_purchase_lh']))

Now we compare this with Singaporeans for which there are 17789 in the data and 710 of them bought an insurance within the last 3 months

In [ ]:
print(df[df['ctrycode_grouped'] == 'Singapore'].shape[0])
print(sum(df[df['ctrycode_grouped'] == 'Singapore']['f_purchase_lh']))

Below is the proportion of clients by nationality

In [ ]:
categories = ['Malaysians', 'Others', 'Singaporeans']
values_set1 = [df[df['ctrycode_grouped'] == 'Malaysia'].shape[0], df[df['ctrycode_grouped'].apply(lambda x:x not in ['Malaysia','Singapore'])].shape[0], 
               df[df['ctrycode_grouped'] == 'Singapore'].shape[0]] 

data = pd.DataFrame({'Nationality': categories, 'Value': values_set1})

colors = ['skyblue', 'salmon', 'lightgreen']

plt.figure(figsize=(8, 6))
ax = sns.barplot(x='Nationality', y='Value', data=data, palette=colors)

for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()-50),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points', fontsize=12)

plt.title('Proportion of clients by nationality')
plt.ylabel('Value')
plt.show()

Out of everyone in the client list, only Singaporeans bought insurance

In [ ]:
categories = ['Malaysians', 'Others', 'Singaporeans']
values_set1 = [sum(df[df['ctrycode_grouped'] == 'Malaysia']['f_purchase_lh']), sum(df[df['ctrycode_grouped'].apply(lambda x:x not in ['Malaysia','Singapore'])]['f_purchase_lh']), 
               sum(df[df['ctrycode_grouped'] == 'Singapore']['f_purchase_lh'])]

data = pd.DataFrame({'Nationality': categories, 'Value': values_set1})

colors = ['skyblue', 'salmon', 'lightgreen']

plt.figure(figsize=(8, 6))
ax = sns.barplot(x='Nationality', y='Value', data=data, palette=colors)

for p in ax.patches:
    ax.annotate(f'{p.get_height()}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points', fontsize=12)
plt.ylim(0, max(data['Value']) * 1.1)
plt.title('Proportion of clients by nationality who bought insurance within the last 3 months')
plt.ylabel('Value')
plt.show()

For race, we filled the missing values with others

In [ ]:
df['race_desc'] = df['race_desc'].fillna('Others')
df['race_desc'].value_counts()

Since there are only 2 values for sex we filled the missing values with the most common gender which happens to be male.

In [ ]:
df['cltsex_fix'] = df['cltsex_fix'].fillna(df['cltsex_fix'].mode()[0])
df['cltsex_fix'].value_counts()

We have the flag columns which are binary variables which indicates the various risk and status indicators

In [ ]:
flag_columns = [
    'flg_substandard',
    'flg_is_borderline_standard',
    'flg_is_revised_term',
    'flg_is_rental_flat',
    'flg_has_health_claim',
    'flg_has_life_claim',
    'flg_gi_claim',
    'flg_is_proposal',
    'flg_with_preauthorisation',
    'flg_is_returned_mail'
]

We then sum up the risk indicators

In [ ]:
df[flag_columns] = df[flag_columns].fillna(0)

In [ ]:
df['flag_sum'] = df[flag_columns].sum(axis=1)

In [ ]:
df=df.drop(columns=flag_columns)

Similarly we sum up the consent to getting contacted and valid contact points

In [ ]:
consent_columns = [
    'is_consent_to_mail',
    'is_consent_to_email',
    'is_consent_to_call',
    'is_consent_to_sms'
]

validity_columns = [
    'is_valid_dm',
    'is_valid_email'
]

In [ ]:
df[consent_columns] = df[consent_columns].fillna(0)
df[validity_columns] = df[validity_columns].fillna(0)

In [ ]:
df['communication_consent_sum'] = df[consent_columns].sum(axis=1)
df['communication_validity_sum'] = df[validity_columns].sum(axis=1)

In [ ]:
df=df.drop(columns=[*consent_columns,*validity_columns])

In [ ]:
df['hh_size_est'] = df['hh_size_est'].replace('>4', '5')
df['hh_size_est'] = pd.to_numeric(df['hh_size_est'])

Then for the annual income of clients, we filled the missing values with 0 and Below 30K with 1 etc with the purpose of creating it into ordinal values. Which means that they have an order to it and 5 is the highest amount while 0 is the lowest.

In [ ]:
df['annual_income_est'] = df['annual_income_est'].fillna(0)
income_ranges = {
    0:0,
    'E.BELOW30K': 1,
    'D.30K-60K': 2,
    'C.60K-100K': 3,
    'B.100K-200K': 4,
    'A.ABOVE200K': 5
}
df['annual_income_est'] = df['annual_income_est'].apply(lambda x: income_ranges[x])


We also have other categorical variables such as housewife retiree, is sg pr, is class 1 or 2 and dependent in at least another policy. We fill these missing values with the most common values found in the column

In [ ]:
columns_to_fill_median = ['is_housewife_retiree', 'is_sg_pr', 'is_class_1_2', 'is_dependent_in_at_least_1_policy']

# Fill NaN values with the median in the specified columns (If there are more 1 than 0 then the value will become 1)
df[columns_to_fill_median] = df[columns_to_fill_median].fillna(df[columns_to_fill_median].median())

We then fill the 

In [ ]:
df['hh_size'] .fillna(df['hh_size'].mean())

Here we have hh_20 and pop_20, we keep both and removed hh_size and hh_size_est as the 2 latter can be calculated from the first two columns

In [ ]:
columns_to_fill_median = ['hh_20', 'pop_20']
column_to_remove = ['hh_size_est', 'hh_size']
df[columns_to_fill_median] = df[columns_to_fill_median].apply(pd.to_numeric, errors='coerce')
df[columns_to_fill_median] = df[columns_to_fill_median].fillna(df[columns_to_fill_median].median(skipna=True))
df=df.drop(columns=column_to_remove)


In [ ]:
filtered_columns = df.filter(like='ape_').columns.tolist() + \
                    df.filter(like='sumins_').columns.tolist() + \
                    df.filter(like='prempaid_').columns.tolist()

df[filtered_columns] = df[filtered_columns].fillna(0)

We sum up the annual premium equivalent, sum insured and premium paid together

In [ ]:
prefixes=['ape','sumins','prempaid']
for prefix in prefixes:
    columns_to_sum = df.filter(like=f'{prefix}').columns
    df[f'sum_{prefix}'] = df[columns_to_sum].astype(float).sum(axis=1)

In [ ]:
df=df.drop(columns=filtered_columns)

In [ ]:
metric_columns = [
    'n_months_last_bought_products',
    'flg_latest_being_lapse',
    'flg_latest_being_cancel',
    'recency_lapse',
    'recency_cancel',
    'tot_inforce_pols',
    'tot_cancel_pols',
    'f_ever_declined_la'
]

In [ ]:
for col in metric_columns:
    print(f"NaN counts for {col}: {df[col].isna().sum()}")

We remove number of months last bought (product code) as there were too many features to keep and n_months_last_bought_products is concise enough to tell us how many months ago did the client purchase an insurance

In [ ]:
filtered_columns_last_bought = df.filter(like = 'n_months_last_bought').drop(columns = 'n_months_last_bought_products').columns.tolist()
filtered_columns_last_bought

In [ ]:
df = df.drop(columns=filtered_columns_last_bought)

We filled the missing values in affconnect with 0

In [ ]:
flags=['flg_affconnect_show_interest_ever', 'flg_affconnect_ready_to_buy_ever',
       'flg_affconnect_lapse_ever']
for f in flags:
    print(df[f].value_counts())

In [ ]:
df[flags]=df[flags].fillna(0)

We sum up the gi_claim_ever but since there were no claims for any clients we subsequently removed it

In [ ]:
columns_to_sum = ['flg_gi_claim_29d435_ever', 'flg_gi_claim_058815_ever', 'flg_gi_claim_42e115_ever', 'flg_gi_claim_856320_ever']
df['sum_gi_claim'] = df[columns_to_sum].sum(axis=1, skipna=True)
df['sum_gi_claim'] = df['sum_gi_claim'].fillna(0)

df=df.drop(columns=columns_to_sum)

In [ ]:
df=df.drop(columns=['sum_gi_claim'])

We turn affcon_visit_days into a boolean of whether a person visited.

In [ ]:
df['affcon_visited'] = df['affcon_visit_days'].fillna(0).apply(lambda x: int(x))
df=df.drop(columns='affcon_visit_days')

In [ ]:
cancel_cols=['f_ever_declined_la','tot_cancel_pols','recency_cancel','recency_lapse']
df[cancel_cols]=df[cancel_cols].fillna(0)

We sum up ever bought.

In [ ]:
filter_f_ever_bought = df.filter(like='f_ever_bought').columns.to_list()
df['sum_f_ever_bought']=df[filter_f_ever_bought].sum(axis=1)


In [ ]:

df=df.drop(columns=filter_f_ever_bought)

We now sum up the success claims and the unsuccessful claims



In [ ]:
claim_cnt_success = ['giclaim_cnt_success','hlthclaim_cnt_success']
df['claim_cnt_success'] = df[claim_cnt_success].fillna(0).sum(axis=1, skipna=True)
df=df.drop(columns=claim_cnt_success)
claim_cnt_unsuccess = ['hlthclaim_cnt_unsuccess', 'giclaim_cnt_unsuccess']
df['claim_cnt_unsuccess'] = df[claim_cnt_unsuccess].fillna(0).sum(axis=1, skipna=True)
df=df.drop(columns=claim_cnt_unsuccess)

For the remaining columns, we drop it to minimize the features added to the model to reduce the overfitting.

In [ ]:
columns_n_months_since_lapse = df.filter(like='n_months_since_lapse').columns.to_list()
df=df.drop(columns=columns_n_months_since_lapse)

In [ ]:
na_columns = df.columns[df.isna().any()]

print(na_columns)

In [ ]:
df=df.drop(columns=na_columns)

## Clustering and data visualisation

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

We performed a Principal component analysis and showed the clusterings in 3D to see if there are clear outliers or if the clusters are close together

In [ ]:
scaler = StandardScaler()
scaled_features = scaler.fit_transform(df.drop(columns=df.select_dtypes(include=['string','object'])))
num_clusters = 3


kmeans = KMeans(n_clusters=num_clusters)


kmeans.fit(scaled_features)


cluster_labels = kmeans.labels_

pca = PCA(n_components=3)
reduced_features = pca.fit_transform(scaled_features)

explained_variance_ratio = pca.explained_variance_ratio_

for i in range(3):
    print(f"Explained Variance Ratio for PC{i+1}: {explained_variance_ratio[i]*100:.2f}%")
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

scatter = ax.scatter(reduced_features[:, 0], reduced_features[:, 1], reduced_features[:, 2], c=cluster_labels, cmap='viridis')

# Adding colorbar
cbar = plt.colorbar(scatter)
cbar.set_label('Cluster Labels')

ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')

plt.title('3D Scatter Plot of Clusters')

plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
colors = np.where(df["f_purchase_lh"] == 1, 'red', 'green')

alpha = np.where(df["f_purchase_lh"] == 1, 1.0, 0.1)

ax.scatter(reduced_features[:, 0], reduced_features[:, 1], reduced_features[:, 2], c=colors, alpha=alpha, s=1)

ax.set_title('3D PCA for labels')
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
ax.set_zlabel('Principal Component 3')

plt.show()


In [ ]:
X=df.drop(columns=["f_purchase_lh"])
y=df["f_purchase_lh"]

In [ ]:
bool_columns = X.columns[(X.eq(0) | X.eq(1)).all()]
X[bool_columns] = X[bool_columns].astype(bool)
categorical_features=X.select_dtypes(include=['object','string']).columns.tolist()
categorical_features

In [ ]:
cat_feat=[*categorical_features,*bool_columns]

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import precision_recall_curve ,auc,roc_auc_score

We used Synthetic Minority Oversampling Technique (smote) to create synthethic data to form a more balanced dataset and in particular we used smoteNC instead of smote for categoical features as interpolating binary variables does not work.

## Precision-Recall Threshold
First, we have to find a lambda to act as the threshold.
Given that the challenge is to dissect the dataset to uncover the critical touchpoints that contribute to customer drop-off, we want to reduce the number of false positives (i.e Number of customers who do not buy but we predict that they buy) to reduce customer churn while also balancing false negatives (i.e Number of customers who buy but we predict that they do not buy) to reduce inefficient marketing.
Thus we use f1 score to devise a harmonic mean of precision and recall.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, y_train = SMOTENC(categorical_features=cat_feat).fit_resample(X_train, y_train)
train_pool = Pool(X_train, label=y_train,cat_features=cat_feat)
test_pool = Pool(X_val, label=y_val,cat_features=cat_feat)

model = CatBoostClassifier(iterations=500, depth=8, learning_rate=0.1, loss_function='Logloss', verbose=False,cat_features=cat_feat)

model.fit(train_pool, eval_set=test_pool)


In [ ]:
y_val_proba = model.predict_proba(X_val)[:, 1]

precision, recall, thresholds = precision_recall_curve(y_val, y_val_proba)

auc_pr = auc(recall, precision)
f1_scores = 2 * (precision * recall) / (precision + recall)
optimal_threshold_index = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_threshold_index]

plt.figure(figsize=(8, 6))
plt.plot(recall, precision, label=f'AUC-PR = {auc_pr:.2f}')
plt.scatter(recall[optimal_threshold_index], precision[optimal_threshold_index], marker='o', color='r', label='Optimal Threshold')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.show()
y_val_proba = model.predict_proba(X_val)[:, 1]

roc_auc = roc_auc_score(y_val, y_val_proba)
print(f'Optimal Threshold: {optimal_threshold}')

We used StratifiedKFold for imbalanced datasets because it maintains the original class distribution in each cross-validation fold. This ensures a more accurate representation during model evaluation, enhancing the reliability of the results. It helps the model generalize better across different class imbalances in the dataset.

In [ ]:
# Number of folds
n_splits = 5
# Initialise Stratified KFold
stratified_kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
models_dict = {}

In [ ]:
for fold,(train_index, test_index) in enumerate(stratified_kf.split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
    X_train, y_train = SMOTENC(categorical_features=cat_feat).fit_resample(X_train, y_train)
    train_pool = Pool(X_train, label=y_train,cat_features=cat_feat)
    test_pool = Pool(X_val, label=y_val,cat_features=cat_feat)

    model = CatBoostClassifier(iterations=500, depth=8, learning_rate=0.1, loss_function='Logloss', verbose=False,cat_features=cat_feat)
    model.fit(train_pool, eval_set=test_pool)
    models_dict[f"Fold_{fold+1}"] = model

In [ ]:
def predict(models, X_input, cat_feat,thres=0.5):
    input_pool = Pool(X_input, cat_features=cat_feat)
    predictions = [ model.predict(input_pool) for model in models.values()]
    average_prediction = sum(predictions) / len(predictions)

    return average_prediction >= thres

In [ ]:
def testing_hidden_data(hidden_data: pd.DataFrame) -> list:
    df=hidden_data
    df.replace('None',None,inplace=True)
    df['adjusted_first_purchase'] = (datetime.datetime.now()-pd.to_datetime(df['min_occ_date']).fillna(pd.to_datetime(df['min_occ_date']).median())).dt.days/365
    df['adjusted_dob'] = (datetime.datetime.now()-pd.to_datetime(df['cltdob_fix']).fillna(pd.to_datetime(df['cltdob_fix']).median())).dt.days/365
    df['client_first_purchase_age']=df['adjusted_dob']-df['adjusted_first_purchase']
    df['ctrycode_desc']=df['ctrycode_desc'].fillna('Singapore')
    df['race_desc'] = df['race_desc'].fillna('Others')
    
    keys_to_drop = ['clntnum', 'min_occ_date', 'cltdob_fix']
    df = df.drop(columns=keys_to_drop)
    
    flag_columns = [
        'flg_substandard',
        'flg_is_borderline_standard',
        'flg_is_revised_term',
        'flg_is_rental_flat',
        'flg_has_health_claim',
        'flg_has_life_claim',
        'flg_gi_claim',
        'flg_is_proposal',
        'flg_with_preauthorisation',
        'flg_is_returned_mail'
    ]
    
    df[flag_columns] = df[flag_columns].fillna(0)
    df['flag_sum'] = df[flag_columns].sum(axis=1)
    df=df.drop(columns=flag_columns)
    df['cltsex_fix'] = df['cltsex_fix'].fillna(df['cltsex_fix'].mode()[0])
    consent_columns = [
    'is_consent_to_mail',
    'is_consent_to_email',
    'is_consent_to_call',
    'is_consent_to_sms'
    ]

    validity_columns = [
        'is_valid_dm',
        'is_valid_email'
    ]
    
    df[consent_columns] = df[consent_columns].fillna(0)
    df[validity_columns] = df[validity_columns].fillna(0)
    df['communication_consent_sum'] = df[consent_columns].sum(axis=1)
    df['communication_validity_sum'] = df[validity_columns].sum(axis=1)
    df=df.drop(columns=[*consent_columns,*validity_columns])
    claim_cnt_success = ['giclaim_cnt_success','hlthclaim_cnt_success']
    df['claim_cnt_success'] = df[claim_cnt_success].fillna(0).sum(axis=1, skipna=True)
    df=df.drop(columns=claim_cnt_success)
    claim_cnt_unsuccess = ['hlthclaim_cnt_unsuccess', 'giclaim_cnt_unsuccess']
    df['claim_cnt_unsuccess'] = df[claim_cnt_unsuccess].fillna(0).sum(axis=1, skipna=True)
    df=df.drop(columns=claim_cnt_unsuccess)
    col_to_drop = ['flg_gi_claim_29d435_ever', 'flg_gi_claim_058815_ever', 'flg_gi_claim_42e115_ever', 'flg_gi_claim_856320_ever']
    df=df.drop(columns=col_to_drop)
    
    
    columns_n_months_since_lapse = df.filter(like='n_months_since_lapse').columns.to_list()
    df=df.drop(columns=columns_n_months_since_lapse)
    
    size_drop = ['hh_size_est', 'hh_size']
    
    df=df.drop(columns=size_drop)

    df['annual_income_est'] = df['annual_income_est'].fillna(0)
    
    income_ranges = {
        0:0,
        'E.BELOW30K': 1,
        'D.30K-60K': 2,
        'C.60K-100K': 3,
        'B.100K-200K': 4,
        'A.ABOVE200K': 5
    }
    
    df['annual_income_est'] = df['annual_income_est'].apply(lambda x: income_ranges[x])
    columns_to_fill_median = ['is_housewife_retiree', 'is_sg_pr', 'is_class_1_2', 'is_dependent_in_at_least_1_policy','hh_20', 'pop_20']
    df[columns_to_fill_median] = df[columns_to_fill_median].apply(pd.to_numeric, errors='coerce')
    df[columns_to_fill_median] = df[columns_to_fill_median].fillna(df[columns_to_fill_median].median())
    df[columns_to_fill_median] = df[columns_to_fill_median].fillna(df[columns_to_fill_median].median(skipna=True))
    filtered_columns = df.filter(like='ape_').columns.tolist() + \
                    df.filter(like='sumins_').columns.tolist() + \
                    df.filter(like='prempaid_').columns.tolist()
    df[filtered_columns] = df[filtered_columns].fillna(0)
    prefixes=['ape','sumins','prempaid']
    for prefix in prefixes:
        columns_to_sum = df.filter(like=f'{prefix}').columns
        df[f'sum_{prefix}'] = df[columns_to_sum].astype(float).sum(axis=1)
    df=df.drop(columns=filtered_columns)
    flags=['flg_affconnect_show_interest_ever', 'flg_affconnect_ready_to_buy_ever',
       'flg_affconnect_lapse_ever']
    df[flags]=df[flags].fillna(0)

    df['affcon_visited'] = df['affcon_visit_days'].fillna(0).apply(lambda x: int(x))
    df=df.drop(columns='affcon_visit_days')
    cancel_cols=['f_ever_declined_la','tot_cancel_pols','recency_cancel','recency_lapse']
    df[cancel_cols]=df[cancel_cols].fillna(0)
    df['ctrycode_grouped'] = df['ctrycode_desc'].apply(lambda x: x if x in ['Singapore', 'Malaysia'] else 'Others')
    na_columns = df.columns[df.isna().any()]
    df=df.drop(columns=na_columns)
    
    filter_f_ever_bought = df.filter(like='f_ever_bought').columns.to_list()
    df['sum_f_ever_bought']=df[filter_f_ever_bought].sum(axis=1)
    df=df.drop(columns=filter_f_ever_bought)
    
    bool_columns = df.columns[(df.eq(0) | df.eq(1)).all()]
    df[bool_columns] = df[bool_columns].astype(bool)
    categorical_features=df.select_dtypes(include=['object','string']).columns.tolist()
    cat_feat=[*categorical_features,*bool_columns]
    result = predict(models_dict,df,cat_feat,optimal_threshold)
    return result

In [ ]:
test_df = pd.read_parquet(filepath)
try:
    test_df = test_df.drop(columns=["f_purchase_lh"])
except Exception as e:
    print(e)
print(testing_hidden_data(test_df))